In [ ]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 22.1 MB/s eta 0:00:00


In [ ]:
import re
import sys
import os
from Bio import SeqIO
from Bio.Seq import Seq

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

# Replace 'Your Folder Name' with the actual name of your folder in Google Drive
folder_path = '/content/drive/My Drive/tnseq/reference_genomes/MGH_66/'

# Change the current working directory to your folder
os.chdir(folder_path)

# Verify the current working directory
print(f"Current working directory: {os.getcwd()}")

Current working directory: /content/drive/My Drive/tnseq/reference_genomes/MGH_66


In [ ]:
#Check the format of the fna and gff file

fna_file = folder_path+"GCF_000694555.1_Kleb_pneu_MGH_66_V1_genomic.fna"
gff_file = folder_path+"GCF_000694555.1_genomic.gff"

print

try:
    with open(fna_file, 'r') as f:
        for i in range(10): # Read and print the first 10 lines
            line = f.readline()
            if not line: # Stop if end of file is reached before 10 lines
                break
            print(line.strip())
except FileNotFoundError:
    print(f"Error: The file {fna_file} was not found.")
except Exception as e:
    print(f"An error occurred: {e}")


try:
    with open(gff_file, 'r') as f:
        for i in range(10): # Read and print the first 10 lines
            line = f.readline()
            if not line: # Stop if end of file is reached before 10 lines
                break
            print(line.strip())
except FileNotFoundError:
    print(f"Error: The file {gff_file} was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

>NZ_KK737546.1 Klebsiella pneumoniae MGH 66 aedZF-supercont1.1, whole genome shotgun sequence
ATCTGATCCTTCAACTCAGCAAAAGTTCGATTTATTCAACAAAGCCATTCAAAACACCTAAGCATCTCTTACCAGACCCG
GAGCCAGAGATAAAAGTTGTGCATGCTCCAGCTGCATGGAAGACTATTTTCCCATGGGTAAAGAACAGATATTATCGAGA
ATTGCAGCAGGCAGAAGAATCTTTCAATAAAAGTAAGGAAGATCATTCAATTAAACTTTTAAGCCAGAAGGTTGAGCTTG
ATGCTTTAGTTGCTGATTATCAGGCCCGAAGAACTGCTTACCTCGAAGAAATAAAAAGCCAGCACGACGAAGTTGACCAG
TTTGAGCAAGACTACCTCAACTGCGATCCTGACAGCGTATTGGCATATTGCGAAATGGTGCTGACACGGTCTGAATATCC
TGAGAACGGCTTCCCACAAGCCTTTAGGTTGGCCTATTTACCTGACAGTAAGGAGTTGGTGGTTGAATACAACTTACCTG
ATATTGCAGTAGTACCGCGCGAGTTGGAATACAGATATGTCAAAACGAGAGATGCCATTGATGCCAAGGCGCGAAAGATT
GGCGAGATCAAAGAACTCTATCAAAACATAATCGCCGCGATCACACTCCGCACAATGCATGAGCTTTTCGAAGCTGACAA
GGCATCTGCTTTAACATCAGTTCTGTTTAACGGTGTAATCGAAACTATCGATCCTACAAGCGGGCATGACACTAAAGTCA
##gff-version 3
#!gff-spec-version 1.21
#!processor NCBI annotwriter
#!genome-build Kleb_pneu_MGH_66_V1
#!genome-build-accession NCBI_Assembly:GCF_000694555.1
#!annotation-date 

In [ ]:
#Written by Claude

def process_fasta_with_biopython(input_file, output_file):
    """Process FASTA file and write reverse complements using Biopython."""
    try:
        sequences_processed = 0

        with open(output_file, 'w') as output_handle:
            # Parse input FASTA file
            for record in SeqIO.parse(input_file, "fasta"):
                # Create reverse complement using Biopython
                rev_comp_seq = record.seq.reverse_complement()

                # Create new record with modified description
                new_id = f"RevComp_{record.id}"
                new_description = f"reverse complement of {record.description}"

                # Create new SeqRecord
                from Bio.SeqRecord import SeqRecord
                rev_comp_record = SeqRecord(
                    rev_comp_seq,
                    id=new_id,
                    description=new_description
                )

                # Write to output file
                SeqIO.write(rev_comp_record, output_handle, "fasta")
                sequences_processed += 1

        return sequences_processed

    except FileNotFoundError:
        print(f"Error: File '{input_file}' not found.")
        return None
    except Exception as e:
        print(f"Error processing file: {e}")
        return None


In [ ]:
def validate_sequences(input_file):
    """Validate and show information about input sequences."""
    try:
        sequences = list(SeqIO.parse(input_file, "fasta"))

        if not sequences:
            print("No sequences found in the input file.")
            return False

        print(f"Found {len(sequences)} sequence(s):")

        for i, record in enumerate(sequences, 1):
            seq_type = "DNA" if set(str(record.seq).upper()) <= set("ATGCNRYSWKMBDHV") else "Unknown"
            print(f"  {i}. {record.id} - Length: {len(record.seq)} bp - Type: {seq_type}")

            # Show first 50 bases as preview
            preview = str(record.seq)[:50]
            if len(record.seq) > 50:
                preview += "..."
            print(f"     Preview: {preview}")

        return True

    except Exception as e:
        print(f"Error validating sequences: {e}")
        return False

In [ ]:
#Make gene list from gff (written by Claude)
def create_gene_list_from_gff(gff_file, output_file=None):
    """
    Create a gene list file from a GFF file.

    Args:
        gff_file (str): Path to input GFF file
        output_file (str): Path to output gene list file (optional)

    Returns:
        str: Path to created gene list file
    """
    if output_file is None:
        base_name = os.path.splitext(os.path.basename(gff_file))[0]
        output_file = f"{base_name}_genelist.txt"

    genes_found = 0

    try:
        with open(gff_file, 'r') as infile, open(output_file, 'w') as outfile:
            for line in infile:
                line = line.strip()

                # Skip empty lines and comments
                if not line or line.startswith('#'):
                    continue

                # Split GFF line (tab-delimited)
                fields = line.split('\t')

                # GFF format: seqname, source, feature, start, end, score, strand, frame, attribute
                if len(fields) < 9:
                    continue

                seqname = fields[0]     # chromosome/contig
                feature = fields[2]     # feature type
                start = fields[3]       # start position
                end = fields[4]         # end position
                attributes = fields[8]  # attributes field

                # Only process gene features
                if feature.lower() != 'cds':
                    continue

                # Extract gene ID from attributes
                gene_id = extract_gene_id(attributes)

                if gene_id:
                    # Format: chr, gene_id, blank, blank, blank, start, end
                    outfile.write(f"{seqname}\t{gene_id}\t\t\t\t{start}\t{end}\n")
                    genes_found += 1

        print(f"Created gene list: {output_file}")
        print(f"Found {genes_found} genes")
        return output_file

    except FileNotFoundError:
        print(f"Error: GFF file '{gff_file}' not found.")
        return None
    except Exception as e:
        print(f"Error processing GFF file: {e}")
        return None

def extract_gene_id(attributes):
    """
    Extract gene ID from GFF attributes field using the ID tag.
    """
    # Look specifically for Parent= tag
    match = re.search(r'locus_tag=([^;]+)', attributes)
    if match:
        gene_id = match.group(1).strip('"')
        return gene_id

    # If no ID found, return None
    return None



In [ ]:
#Run functions to create reverse complement .fna

input_file = fna_file  # Change this to your input file name

# Generate output filename automatically
base_name = os.path.splitext(os.path.basename(input_file))[0]
output_file = f"{base_name}_reverse_complement.fna"

if(not os.path.exists(output_file)):
  # Validate input sequences
  if validate_sequences(input_file):
    print("-" * 50)
    print("Processing sequences...")

    # Process the file
    sequences_processed = process_fasta_with_biopython(input_file, output_file)

    if sequences_processed is not None:
        print(f"Successfully processed {sequences_processed} sequence(s)")
        print(f"Reverse complement sequences written to: {output_file}")
        print("Done!")
    else:
        print("Processing failed.")
  else:
    print("Validation failed - check your input file.")
else:
   print(output_file + "already exists. Not overwritting.")

GCF_000694555.1_Kleb_pneu_MGH_66_V1_genomic_reverse_complement.fnaalready exists. Not overwritting.


In [23]:
#Run function to make gene list
input_gff = gff_file  # Change this to your input file name

# Generate output filename automatically
base_name = os.path.splitext(os.path.basename(input_gff))[0]
output_gff = f"{base_name}_gene_list.txt"

# Check if the file exists and is empty before skipping
if os.path.exists(output_gff) and os.path.getsize(output_gff) > 0:
  print(output_gff + " already exists and is not empty. Not overwritting.")
else:
  create_gene_list_from_gff(input_gff, output_gff)

Created gene list: GCF_000694555.1_genomic_gene_list.txt
Found 4826 genes
